## Vaccine Efficacy

In [ ]:
import math
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt

We have the following data from the clinical trials of the three vaccines.

In [ ]:
n_Pfizer_vaccine = 8
n_Pfizer_placebo = 162

n_Pfizer_total = 21500

Pfizer_vaccine_outcomes = np.concatenate([np.zeros(n_Pfizer_total - n_Pfizer_vaccine), np.ones(n_Pfizer_vaccine)])
Pfizer_placebo_outcomes = np.concatenate([np.zeros(n_Pfizer_total - n_Pfizer_placebo), np.ones(n_Pfizer_placebo)])


n_Moderna_vaccine = 5
n_Moderna_placebo = 90

n_Moderna_total = 15000

Moderna_vaccine_outcomes = np.concatenate([np.zeros(n_Moderna_total - n_Moderna_vaccine), np.ones(n_Moderna_vaccine)])
Moderna_placebo_outcomes = np.concatenate([np.zeros(n_Moderna_total - n_Moderna_placebo), np.ones(n_Moderna_placebo)])


n_AstraZeneca_1_vaccine = 3
n_AstraZeneca_1_placebo = 34

n_AstraZeneca_1_total = 1371

AstraZeneca_1_vaccine_outcomes = np.concatenate([np.zeros(n_AstraZeneca_1_total - n_AstraZeneca_1_vaccine), np.ones(n_AstraZeneca_1_vaccine)])
AstraZeneca_1_placebo_outcomes = np.concatenate([np.zeros(n_AstraZeneca_1_total - n_AstraZeneca_1_placebo), np.ones(n_AstraZeneca_1_placebo)])


n_AstraZeneca_2_vaccine = 26
n_AstraZeneca_2_placebo = 68

n_AstraZeneca_2_total = 2248

AstraZeneca_2_vaccine_outcomes = np.concatenate([np.zeros(n_AstraZeneca_2_total - n_AstraZeneca_2_vaccine), np.ones(n_AstraZeneca_2_vaccine)])
AstraZeneca_2_placebo_outcomes = np.concatenate([np.zeros(n_AstraZeneca_2_total - n_AstraZeneca_2_placebo), np.ones(n_AstraZeneca_2_placebo)])

We can build a probablistic model to describe this noisy measurement process.

In [ ]:
model = pm.Model()

with model:
    
    # Probability of exposure

    p_Pfizer = pm.Beta('p_Pfizer', alpha=1, beta=1)
    p_Moderna = pm.Beta('p_Moderna', alpha=1, beta=1)
    p_AstraZeneca_1 = pm.Beta('p_AstraZeneca_1', alpha=1, beta=1)
    p_AstraZeneca_2 = pm.Beta('p_AstraZeneca_2', alpha=1, beta=1)
    
    # Probability of protection

    e_Pfizer = pm.Beta('e_Pfizer', alpha=1, beta=1)
    e_Moderna = pm.Beta('e_Moderna', alpha=1, beta=1)
    e_AstraZeneca_1 = pm.Beta('e_AstraZeneca_1', alpha=1, beta=1)
    e_AstraZeneca_2 = pm.Beta('e_AstraZeneca_2', alpha=1, beta=1)
    
    # Likelihood of observed number of positive tests
    
    like_Pfizer_vaccine = pm.Bernoulli('like_Pfizer_vaccine', p=p_Pfizer * (1.0 - e_Pfizer), observed=Pfizer_vaccine_outcomes)
    like_Pfizer_placebo = pm.Bernoulli('like_Pfizer_placebo', p=p_Pfizer, observed=Pfizer_placebo_outcomes)

    like_Moderna_vaccine = pm.Bernoulli('like_Moderna_vaccine', p=p_Moderna * (1.0 - e_Moderna), observed=Moderna_vaccine_outcomes)    
    like_Moderna_placebo = pm.Bernoulli('like_Moderna_placebo', p=p_Moderna, observed=Moderna_placebo_outcomes)
    
    like_AstraZeneca_1_vaccine = pm.Bernoulli('like_AstraZeneca_1_vaccine', p=p_AstraZeneca_1 * (1.0 - e_AstraZeneca_1), observed=AstraZeneca_1_vaccine_outcomes)
    like_AstraZeneca_1_placebo = pm.Bernoulli('like_AstraZeneca_1_placebo', p=p_AstraZeneca_1, observed=AstraZeneca_1_placebo_outcomes)
    
    like_AstraZeneca_2_vaccine = pm.Bernoulli('like_AstraZeneca_2_vaccine', p=p_AstraZeneca_2 * (1.0 - e_AstraZeneca_2),  observed=AstraZeneca_2_vaccine_outcomes)
    like_AstraZeneca_2_placebo = pm.Bernoulli('like_AstraZeneca_2_placebo', p=p_AstraZeneca_2, observed=AstraZeneca_2_placebo_outcomes)


We run the model with some default settings.

In [ ]:
with model:
    
    idata = pm.sample(2000)

We can check the trace using the inbuilt plotting tool.

In [ ]:
pm.plot_trace(idata, var_names=['e_Pfizer', 'e_Moderna', 'e_AstraZeneca_1', 'e_AstraZeneca_2'], figsize=(12,16), combined=False);

Finally when we are happy we can generate some final plots and report on the results.

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(12, 9))

data = idata.posterior.get('e_Pfizer')[0];

axes[0].hist(data, histtype='stepfilled', bins=100, alpha=0.85, color="#467821", density=True)
axes[0].set_xlim(0, 1)
axes[0].set_title('Pfizer')
axes[0].set_xlabel('Probability of protection')

data = idata.posterior.get('e_Moderna')[0];

axes[1].hist(data, histtype='stepfilled', bins=100, alpha=0.85, color="#A60628", density=True)
axes[1].set_xlim(0, 1)
axes[1].set_title('Moderna')
axes[1].set_xlabel('Probability of protection')

data = idata.posterior.get('e_AstraZeneca_1')[0];

axes[2].hist(data, histtype='stepfilled', bins=100, alpha=0.85, color="#784621", density=True)
axes[2].set_xlim(0, 1)
axes[2].set_title('AstraZeneca Regime 1')
axes[2].set_xlabel('Probability of protection')

data = idata.posterior.get('e_AstraZeneca_2')[0];

axes[3].hist(data, histtype='stepfilled', bins=100, alpha=0.85, color="#06A628", density=True)
axes[3].set_xlim(0, 1)
axes[3].set_title('AstraZeneca Regime 2')
axes[3].set_xlabel('Probability of protection')

plt.tight_layout()

plt.savefig('results.pdf')